<a href="https://colab.research.google.com/github/Tony6512/Kaggle-Datasets/blob/main/ford_car_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

https://www.kaggle.com/datasets/adhurimquku/ford-car-price-prediction/data

# Intro

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import scipy
import statsmodels.api as sm
import time

# visualization
import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/kaggle/input/ford-car-price-prediction/ford.csv")

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Strip extra spaces from all string entries in the DataFrame
# df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

No missing data.

In [ ]:
df.isna().sum()

No NA labels.

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated(keep = False)]

In [ ]:
## remove duplicates
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.describe(exclude = [np.number])

In [ ]:
cts_feat = df.describe().columns
cts_feat

In [ ]:
cts_feat.shape

In [ ]:
cat_feat = df.describe(exclude = [np.number]).columns
cat_feat

In [ ]:
cat_feat.shape

# Data Exploration and Data Preparation

In [ ]:
sns.heatmap(df.select_dtypes(include=[np.number]).corr(), annot=True, square=True, cmap='coolwarm', annot_kws={'size': 8}, fmt = '.2f')
plt.title('Pearson')
plt.show()
sns.heatmap(df.select_dtypes(include=[np.number]).corr(method = 'spearman'), annot=True, square=True, cmap='coolwarm', annot_kws={'size': 8}, fmt = '.2f')
plt.title('Spearman')
plt.show()
sns.heatmap(df.select_dtypes(include=[np.number]).corr(method = 'kendall'), annot=True, square=True, cmap='coolwarm', annot_kws={'size': 8}, fmt = '.2f')
plt.title('Kendall')
plt.show()
None
# Pearson: For linear relationships between continuous variables. linear association
# Spearman: For ranked or ordinal data. monoticic association
# Kendall: For ordinal data, less sensitive to outliers. ordinal association

It seems like year should have a big affect on price.

In [ ]:
# fig, axs = plt.subplots(nrows=2, figsize=(15, 15))
# sns.heatmap(df_train.corr(), ax=axs[0], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 8}, fmt = '.2f')
# sns.heatmap(df_test.corr(), ax=axs[1], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 8}, fmt = '.2f')
# plt.show()

# fig, axs = plt.subplots(nrows=2, figsize=(15, 15))
# sns.heatmap(df_train.select_dtypes(include=[np.number]).corr(), ax=axs[0], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 8}, fmt = '.2f')
# sns.heatmap(df_test.select_dtypes(include=[np.number]).corr(), ax=axs[1], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 8}, fmt = '.2f')
# plt.show()

## Continuous Features

In [ ]:
for feat in cts_feat:
    sns.histplot(df, x = feat, kde = True)
    plt.show()

Some of the later years (such as 2060) are probably typos (more likely 2006).

In [ ]:
df[df['year']==2060]

In [ ]:
df.loc[17726]

In [ ]:
df.loc[df['year']==2060, 'year'] = 2006 ## change 2060 to 2006
df.loc[17726]

Depending on the models used we could transform the target (price) so that it is normal. Here are some qq plots to see how normal the target is.

In [ ]:
sm.qqplot(df['price'], line='s')
plt.title('QQ Plot')
plt.show()
None

In [ ]:
sm.qqplot(np.log(df['price']), line='s')
plt.title('QQ Plot')
plt.show()
None

In [ ]:
sm.qqplot(scipy.stats.boxcox(df['price'])[0], line='s')
plt.title('QQ Plot')
plt.show()
None

## Categorical Features

In [ ]:
for feat in cat_feat:
    if feat == 'model':
        ax = sns.countplot(df, x = feat,
            order = df[feat].value_counts().sort_values(ascending=False).index)
        # ax.bar_label(ax.containers[0])
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        ax = sns.countplot(df, x = feat,
            order = df[feat].value_counts().sort_values(ascending=False).index)
        ax.bar_label(ax.containers[0])
        plt.show()

Interesting that there are so many manual transmission vehicles. Could this by a typo? One would expect lots of manual cars in older models, however, the min year is 1996 and median of 2017.

## Price vs Continuous Feature

In [ ]:
for feat in cts_feat:
    if feat != 'price':
        sns.scatterplot(df, x = feat, y = 'price', alpha = 0.3)
        plt.show()

201.8 mpg seems like a typo.

In [ ]:
df[df['mpg'] > 190]

In [ ]:
df[df['model'] == 'Kuga']
# df[df['model'].str.strip() == 'Kuga'] ## eliminated extra space earlier

In [ ]:
df_Kuga = df[df['model'] == 'Kuga']
sns.histplot(df_Kuga, x = 'mpg', kde = True)
plt.show()

In [ ]:
df_Kuga.describe()

Unfortunately, 201.8 -> 21.8 nor 201.8 -> 20.8 probabily do not make sense as the min is 27.4. Though the max value could be replaced by the median, I don't think that was the intended value, so we elect to drop the five rows.

In [ ]:
df.drop(df[df['mpg']>190].index, inplace = True)

In [ ]:
df[df['mpg'] > 190]   ## check to make sure rows are dropped

In [ ]:
for feat in cts_feat:
    if feat != 'price':
        sns.histplot(df, x = feat, y = 'price')
        plt.show()

It seems that year and mileage are associated with price. It makes sense.

## Price vs Categorical Feature

In [ ]:
for feat in cat_feat:
    if feat == 'model':
        sns.boxplot(df, x = feat, y = 'price')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        sns.boxplot(df, x = feat, y = 'price')
        plt.show()

Hybrid cars tend to cost more than diesel and petrol. Model of car does affect the price (espicially if it is a mustang).

## Split Data and Data (Feature) Tranformation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
from sklearn.metrics import roc_curve, auc, r2_score, mean_squared_error
SEED = 0

In [ ]:
y = df['price']
X = df.drop(['price'], axis = 1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=SEED)

In [ ]:
cts_feat = X.describe().columns ## exlude target (price)
cts_feat

In [ ]:
ct = ColumnTransformer(
    [("scaling", MinMaxScaler(), cts_feat),
     ("onehot", OneHotEncoder(handle_unknown='ignore'), cat_feat)])

In [ ]:
X_train = ct.fit_transform(X_train)

In [ ]:
ct.get_feature_names_out()

In [ ]:
ct.get_feature_names_out().shape

In [ ]:
X_train.shape

In [ ]:
X_test = ct.transform(X_test)

# Single Model

In [ ]:
from sklearn.ensemble import (RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor,
BaggingRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor)
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
def time_perf(model, X_train, y_train):
    tic = time.perf_counter()
    model.fit(X_train, y_train)
    toc = time.perf_counter()
    elapsed_time = toc = tic
    print(f'{model} elapsed time: {(toc-tic):.2f} [sec]')
    print(f'{model} elapsed time: {((toc-tic)/60):.2f} [min]')
    return model

In [ ]:
rf = RandomForestRegressor()


param_dist = {'n_estimators': [100,250], 'max_depth':[3,6,9,12]}

rf_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                               n_iter = 5, cv=3, verbose = 4, random_state=SEED)


rf_search.fit(X_train, y_train)


y_pred = rf_search.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
rf_search.best_estimator_

In [ ]:
r2

In [ ]:
mse

In [ ]:
normalized_mse = (mse / (np.mean(y_test) ** 2))
normalized_mse

In [ ]:
# Calculate residuals
residuals = y_test - y_pred

# Residual plot
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, color='blue')
plt.axhline(0, color='red', linestyle='--', lw=2)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid()
plt.show()

In [ ]:
# from sklearn.model_selection import learning_curve

# # Generate learning curve data
# train_sizes, train_scores, test_scores = learning_curve(
#     rf_search.best_estimator_, X_train, y_train,
#     train_sizes=np.linspace(0.1, 1.0, 7),
#     cv=3,
#     scoring='neg_mean_squared_error')

# # Calculate mean and standard deviation
# train_scores_mean = -np.mean(train_scores, axis=1)
# test_scores_mean = -np.mean(test_scores, axis=1)

# # Plot learning curve
# plt.figure(figsize=(10, 6))
# plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training MSE')
# plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Validation MSE')
# plt.xlabel('Training Size')
# plt.ylabel('Mean Squared Error')
# plt.title('Learning Curve')
# plt.legend(loc='best')
# plt.grid()
# plt.show()


# Ensemble  Model (Stacking)

In [ ]:
estimators = [('rf', RandomForestRegressor()),
              ('gb', GradientBoostingRegressor())]
parameters = {'rf': {'n_estimators': [100,250], 'max_depth':[3,6,9,12]},
              'gb':{'n_estimators':[100, 150, 200]} }
best_estimators = [0]*len(estimators)

In [ ]:
for i, (name, est) in enumerate(estimators):
    est_search = RandomizedSearchCV(est, parameters[name],
                                    n_iter = 5, cv = 3, verbose = 4).fit(X_train, y_train)
    best_estimators[i] = (name, est_search.best_estimator_)

In [ ]:
## RidgeCV default for meta estimator
est_stacking = StackingRegressor(estimators = best_estimators, final_estimator = None, cv = 5, verbose = 4)

In [ ]:
est_stacking.fit(X_train, y_train)

In [ ]:
est_stacking.score(X_train, y_train)

In [ ]:
y_pred = est_stacking.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
# est_stacking.best_estimator_

In [ ]:
r2

In [ ]:
mse

In [ ]:
normalized_mse = (mse / (np.mean(y_test) ** 2))
normalized_mse

In [ ]:
# Calculate residuals
residuals = y_test - y_pred

# Residual plot
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, color='blue')
plt.axhline(0, color='red', linestyle='--', lw=2)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid()
plt.show()

## Save model

In [ ]:
## for scikit learn
import pickle
model = est_stacking
## Save the model
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

# ## Load the model
# with open('model.pkl', 'rb') as file:
#     loaded_model = pickle.load(file)


In [ ]:
## for tensorflow/keras

# # Save the entire model
# model.save('model.h5')

# # Load the model
# from tensorflow.keras.models import load_model
# loaded_model = load_model('model.h5')


# Load model

In [ ]:
## for scikit learn
import pickle

# # Save the model
# with open('model.pkl', 'wb') as file:
#     pickle.dump(model, file)

# Load the model
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
est_stacking = loaded_model

In [ ]:
## for tensorflow/keras

# # Save the entire model
# model.save('model.h5')

# # Load the model
# from tensorflow.keras.models import load_model
# loaded_model = load_model('model.h5')


In [ ]:
y_pred = est_stacking.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
normalized_mse = (mse / (np.mean(y_test) ** 2))
print(f'mse: {mse}')
print(f'r2: {r2}')
print(f'normalized_mse: {normalized_mse}')

We have good r2 and normalized mse scores.

In [ ]:
# Calculate residuals
residuals = y_test - y_pred

# Residual plot
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, color='blue')
plt.axhline(0, color='red', linestyle='--', lw=2)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid()
plt.show()

# Explainability and Discussion

In [ ]:
X_test.shape

In [ ]:
feature_names = ct.get_feature_names_out()
feature_names

In [ ]:
importance_stacking = permutation_importance(estimator = est_stacking, X = X_test.toarray(), y = y_test,
                                         n_repeats = 20, random_state= SEED)

In [ ]:
plt.figure(figsize=(10, 6))
sorted_idx = np.argsort(importance_stacking.importances_mean)
plt.barh(y = np.linspace(0, 1,len(feature_names)) , width = importance_stacking.importances_mean[sorted_idx],
         height = 1 / len(feature_names), tick_label = feature_names[sorted_idx],
     xerr = importance_stacking.importances_std[sorted_idx])
plt.title(f'Permutation Importance (test error)')
plt.tight_layout()
plt.show()

Year is the most important feature in the model. The next three are engine size, mileage, and mpg.

# Neural Network

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import BaggingClassifier

from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner

In [ ]:
# pip install eli5

In [ ]:
# def build_model(hp):
#     """
#     functional (API) model that is wide (simple rules) and deep (complex rules).
#     Note: cannot use selu b/c network contains skip connections.
#     More tuning hyperparameters.
#     """
#     lr = hp.Float('lr', min_value = 1e-4, max_value = 1e-2, sampling = 'log')
#     units = hp.Int("units", min_value=32, max_value=512, step=32, default = 64)
#     hidden_layers = hp.Int("layers", min_value=1, max_value=4, step=1, default = 2)
#     dropout = hp.Float('do', min_value = 0.2, max_value = 0.5, step = 0.05, default = 0.3)

#     initializer = keras.initializers.HeNormal(seed = SEED)
#     regularizer = None #keras.regularizers.L1(l1=0.001)
#     activation = 'relu'  #'relu', 'elu'
#     loss =  'mean_squared_error' # 'mean_absolute_error'  #'mean_squared_error' # 'huber_loss'
#     metrics = [keras.metrics.RootMeanSquaredError()]

#     input = keras.layers.Input(shape=(X_train.shape[1],))
#     BN0 = layers.BatchNormalization()(input)
#     layer_names = [BN0, 'HL1', 'HL2', 'HL3', 'HL4']
#     for layer in range(1,hidden_layers+1):
#         layer_names[layer] = layers.Dense(units, activation = activation,
#                     kernel_initializer = initializer,
#                     kernel_regularizer = regularizer)(layer_names[layer - 1])
#         BN = layers.BatchNormalization()(layer_names[layer])
#         layer_names[layer] = layers.Dropout(dropout)(BN)


#     concat = keras.layers.Concatenate()([input, layer_names[layer]])
#     output = layers.Dense(1, activation = 'linear')(concat)
#     model = keras.models.Model(inputs=[input], outputs=[output])

#     learning_sched = keras.optimizers.schedules.ExponentialDecay(
#         initial_learning_rate = lr,
#         decay_steps=100000,
#         decay_rate=0.96,
#         staircase=True)
#     model.compile(
#         optimizer=keras.optimizers.Nadam(learning_rate= learning_sched),#'Nadam',
#         loss= loss,
#         metrics=metrics,
#     )

#     return model
# # build_model(keras_tuner.HyperParameters())
# # model.summary()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    """
    Functional (API) model that is wide (simple rules) and deep (complex rules).
    Note: cannot use SELU because the network contains skip connections.
    More tuning hyperparameters.
    """
    lr = hp.Float('lr', min_value=1e-4, max_value=1e-2, sampling='log')
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    hidden_layers = hp.Int("layers", min_value=1, max_value=6, step=1)
    dropout = hp.Float('do', min_value=0.2, max_value=0.5, step=0.05)

    initializer = keras.initializers.HeNormal(seed=SEED)  # Use a fixed seed for reproducibility
    regularizer = keras.regularizers.L1(l1=0.001) # None
    activation =  'elu' #'relu'
    loss = 'mean_squared_error'
    metrics = [keras.metrics.RootMeanSquaredError()]

    input_layer = keras.layers.Input(shape=(X_train.shape[1],))

    x = layers.BatchNormalization()(input_layer)
    for layer in range(hidden_layers):
        x = layers.Dense(units, activation=activation,
                         kernel_initializer=initializer,
                        kernel_regularizer = regularizer)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout)(x)

    # Residual connection/skip connection
    x_res = layers.Dense(units, activation=activation, kernel_initializer=initializer)(input_layer)
    x = layers.add([x, x_res])  # Adding the residual connection

    # Concatenate the input with the last hidden layer
    concat = keras.layers.Concatenate()([input_layer, x])
    output = layers.Dense(1, activation='linear')(concat)

    # Create the model
    model = keras.models.Model(inputs=[input_layer], outputs=[output])

    # Learning rate schedule
    learning_sched = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True
    )

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Nadam(learning_rate=learning_sched),
        loss=loss,
        metrics=metrics,
    )

    return model


In [ ]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel= build_model,
    objective='val_loss',
    max_trials=20,
    # if True, overwrite above directory if search is run again - i.e. don't resume
    overwrite = True
    # ,directory='my_dir',  # Specify a directory to save the results
    # project_name='wide_and_deep'
)

In [ ]:
tuner.search_space_summary()

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_loss', ## validation loss
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
    start_from_epoch = 20,
)

In [ ]:
# tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))
tuner.search(X_train, y_train, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
tuner.results_summary()

In [ ]:
# Get the top  models.
models = tuner.get_best_models(num_models=5)
# for model in models:
#     model.build(input_shape = [None, X_train.shape[1]])
#     model.summary()
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
# input_shape = [None, X_train.shape[1]]
best_model.build(input_shape = [None, X_train.shape[1]])
best_model.summary()

In [ ]:
keras.utils.plot_model(best_model, "multi_input_and_output_model.png", show_shapes=True)

## Save Model

In [ ]:
## for tensorflow/keras
model = best_model
## Save the entire model
model.save('model.h5')

# ## Load the model
# from tensorflow.keras.models import load_model
# loaded_model = load_model('model.h5')

## Load Model

In [ ]:
## for tensorflow/keras

# ## Save the entire model
# model.save('model.h5')

# ## Load the model
from tensorflow.keras.models import load_model
loaded_model = load_model('model.h5')
best_model = loaded_model
best_model.summary()

In [ ]:
# Get the top 5 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
# model.fit(X_train, y_train, epochs=1)
# tensorboard_callback = keras.callbacks.TensorBoard(histogram_freq=1)
history = model.fit(
    X_train, y_train,
#     validation_data=(X_val, y_val),
    validation_split = 0.2,
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping]#, tensorboard_callback],
)

In [ ]:
history_df = pd.DataFrame(history.history)

In [ ]:
history_df

In [ ]:
## regularization only on training data => makes training lc larger than val lc
# history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Loss")
history_df.loc[:, ['root_mean_squared_error', 'val_root_mean_squared_error']].plot(title="Metric")
# history_df.loc[:, ['loss']].plot(title="Cross-entropy")
# history_df.loc[:, ['binary_accuracy']].plot(title="Accuracy")

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
normalized_mse = (mse / (np.mean(y_test) ** 2))
print(f'mse: {mse}')
print(f'r2: {r2}')
print(f'normalized_mse: {normalized_mse}')

In [ ]:
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
# from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

In [ ]:
perm = PermutationImportance(model, scoring='neg_mean_squared_error',
                random_state= SEED, n_iter = 20).fit(X_test.toarray(),y_test)

In [ ]:
plt.figure(figsize=(10, 6))
sorted_idx = np.argsort(perm.feature_importances_)
plt.barh(y = np.linspace(0, 1,len(feature_names)) , width = perm.feature_importances_[sorted_idx],
         height = 1 / len(feature_names), tick_label = feature_names[sorted_idx],
         xerr = perm.feature_importances_std_[sorted_idx])
plt.title('Permutation Importance (test error)')
plt.tight_layout()
None

# End